# 필요한 모듈 설치 및 임포트

In [16]:
%pip install selenium==4.8.2 beautifulsoup4 pandas numpy webdriver_manager

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 62.6/62.6 KB 3.3 MB/s eta 0:00:00
     -------------------------------------- 100.4/100.4 KB 6.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\python.exe -m pip install --upgrade pip' command.


In [17]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
print(selenium.__version__)

4.8.2


# Selenium을 이용하여 브라우저 열기, 클릭, 페이지 이동, 닫기

In [18]:
chrome_options = Options()

chrome_options.add_experimental_option("detach", True)
chrome_options.add_experimental_option("excludeSwitches",["enable-logging"])

driver = webdriver.Chrome(options=chrome_options)

# 무신사 코디맵 크롤링

In [27]:
# options = webdriver.ChromeOptions()
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver = webdriver.Chrome(options=chrome_options)

codimap_list = []
url = 'https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1'
driver.get(url)
driver.implicitly_wait(2)
try:
    driver.maximize_window()
except:
    pass

# 페이지를 가장 아래까지 스크롤하는 방법

interval = 1 # 1초에 한번씩 스크롤 내림

# 현재 문서 높이를 가져와서 저장
prev_height = driver.execute_script("return document.body.scrollHeight")

# 반복 수행
while True:
    # 스크롤을 가장 아래로 내림
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

    # 페이지 로딩 대기
    time.sleep(interval)

    # 현재 문서 높이를 가져와서 저장
    curr_height = driver.execute_script("return document.body.scrollHeight")
    if curr_height == prev_height:
        break

    prev_height = curr_height

# 해당 Xpath 정보를 확인할때까지 5초간 기다림, 5초후에도 해당 Xpath를 찾지 못하면 에러 반환
# elem = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[1]/ul/li[40]/a/div/img')))

columns = ['codimap_category', 'codimap_title', 'codimap_date', 'views', 'comment_numbers']         # 카테고리, 이름, 날짜, 조회수, 댓글수
values = []                                             # 정보를 담을 빈 리스트 생성
soup = BeautifulSoup(driver.page_source, 'html.parser')        # 현재 브라우저의 page의 html 정보를 lxml을 이용하여 가져오기
data_rows = soup.find_all('li', attrs={'class':'style-list-item'}) # 해당 조건에 맞는 정보들을 list로 반환

# 각 <li> 태그를 순회하면서 데이터 추출
for i, row in enumerate(data_rows):
    print('{0} {1}번째 코디맵정보 크롤링'.format(0, i+1))
    blank = []

    # 코디맵 카테고리 추출
    codimap_category = row.find('span', attrs={'class':'style-list-information__text'})
    if codimap_category:
        codimap_category = codimap_category.get_text().strip()
        blank.append(codimap_category)
    else:
        blank.append('Something is wrong')
        print('{}번째 코디맵 가져올때 문제발생'.format(i+1))
        continue

    # 코디맵 이름 추출
    codimap_title = row.find('strong', attrs={'class':'style-list-information__title'})
    if codimap_title:
        codimap_title = codimap_title.get_text().strip()
        blank.append(codimap_title)
    else:
        blank.append('Something is wrong')
        print('{}번째 코디맵 가져올때 문제발생'.format(i+1))
        continue

    # 코디맵 날짜 추출
    codimap_date = row.find('span', attrs={'class':'post-information__text'})
    if codimap_date:
        codimap_date = codimap_date.get_text().strip()
        blank.append(codimap_date)
    else:
        blank.append('Something is wrong')
        print('{}번째 코디맵 가져올때 문제발생'.format(i+1))
        continue

    # 코디맵 조회수 추출
    views = row.find_all('span', attrs={'class':'post-information__text'})[1]
    if views:
        views = views.get_text().strip().split()[1]
        blank.append(views)
    else:
        blank.append('Something is wrong')
        print('{}번째 코디맵 가져올때 문제발생'.format(i+1))
        continue

    # 코디맵 댓글수 추출
    comment_numbers = row.find_all('span', attrs={'class':'post-information__text'})[2]
    if comment_numbers:
        comment_numbers = comment_numbers.get_text().strip().split()[1]
        blank.append(comment_numbers)
    else:
        blank.append('Something is wrong')
        print('{}번째 코디맵 가져올때 문제발생'.format(i+1))
        continue
    
    
        

#         image_url = row.find('img', attrs={'class':'Poster__image--d9XTI'})     # 웹툰이름 가져오기
#         if image_url:                                                           # 해당 조건에 맞는 정보가 하나라도 있을 경우
#             image_url = image_url['src']                                        # src attribute의 값을 가져온다
#             if image_url.startswith('//'):                                      # image_url이 https: 로 시작하지 않는 경우
#                 image_url = 'https:' + image_url
#             blank.append(image_url)
#         else:
#             blank.append('Something is wrong')
#             print('{}번째 웹툰이미지 가져올때 문제발생'.format(i+1))
#             continue

    values.append(blank)
    print('---------------------------------------------------')

time.sleep(1)

df = pd.DataFrame(values, columns = columns)
codimap_list.append(df)


0 1번째 코디맵정보 크롤링
---------------------------------------------------
0 2번째 코디맵정보 크롤링
---------------------------------------------------
0 3번째 코디맵정보 크롤링
---------------------------------------------------
0 4번째 코디맵정보 크롤링
---------------------------------------------------
0 5번째 코디맵정보 크롤링
---------------------------------------------------
0 6번째 코디맵정보 크롤링
---------------------------------------------------
0 7번째 코디맵정보 크롤링
---------------------------------------------------
0 8번째 코디맵정보 크롤링
---------------------------------------------------
0 9번째 코디맵정보 크롤링
---------------------------------------------------
0 10번째 코디맵정보 크롤링
---------------------------------------------------
0 11번째 코디맵정보 크롤링
---------------------------------------------------
0 12번째 코디맵정보 크롤링
---------------------------------------------------
0 13번째 코디맵정보 크롤링
---------------------------------------------------
0 14번째 코디맵정보 크롤링
---------------------------------------------------
0 15번째 코디맵정보 크롤링
--------------------------

In [28]:
df_ = pd.concat(codimap_list).reset_index(drop=True) # index 초기화 해줘야함
df_.head(10)

,codimap_category,codimap_title,codimap_date,views,comment_numbers
0,캐주얼,톤온톤 코디,21.07.02,"17,317",50
1,캐주얼,블랙 매니아,21.02.23,"36,356",47
2,캐주얼,오늘의 코디,22.03.22,"13,426",44
3,댄디,추천 댄디 룩,21.09.15,"21,621",32
4,캐주얼,릴랙스 룩,21.02.23,"24,052",25
5,스트릿,힙하다 힙해,21.02.03,"36,109",24
6,댄디,데이트 코디,21.12.24,"10,996",19
7,캐주얼,심플한 코디,21.07.30,"41,376",18
8,댄디,훈훈 남친 룩,21.03.24,"20,025",17
9,댄디,서머 데이트 룩,21.07.26,"8,373",17


In [29]:
df_.shape

(60, 5)

# Dataframe을 csv 파일로 저장

In [30]:
df_.to_csv('무신사_코디맵_크롤링.csv', encoding = 'utf-8-sig') # 한글이 들어간 정보는 글자가 깨질 수 있어 encoding을 다음과 같이 지정하여 저장